# **ANYMAL C WITH PROXIMAL POLICY OPTIMIZATION**


# Installing dependencies

In [ ]:
!pip install mujoco==2.3.1
!pip install wandb==0.13.10
!pip install mediapy==1.1.4
!pip install matplotlib==3.7.1

In [ ]:
# Graphics and plotting.
print('Installing mediapy:')
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy

# Setting up the environment

In [ ]:
!git clone https://github.com/danimatasd/MUJOCO-AIDL.git

In [ ]:
%env MUJOCO_GL=egl

In [ ]:
import numpy as np
import mujoco
import wandb

from typing import Callable, Optional, Union, List
import mediapy as media
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.distributions import MultivariateNormal
from torch.utils.data.sampler import BatchSampler, SubsetRandomSampler

In [ ]:
#Uncomment this and execute if gpu is going to be used in your code
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#print(device)

In [ ]:
PROJECT = "AIDL-PPO-BD"
MUJOCO_STEPS = 5

In [ ]:
wandb.login()

# Model

In [ ]:
class Agent(nn.Module):
    def __init__(self, obs_len, act_len):
        super(Agent, self).__init__()
        
        self.obs_len = obs_len
        self.act_len = act_len

        self.mlp = nn.Sequential(
            nn.Linear(obs_len, 128),
            nn.Tanh(),
            nn.Linear(128, 128),
            nn.Tanh()
        )

        self.actor = nn.Sequential(
            nn.Linear(128,128),
            nn.Tanh(),
            nn.Linear(128,act_len))
        self.critic = nn.Sequential(
            nn.Linear(128,128),
            nn.Tanh(),
            nn.Linear(128,1))

    def forward(self, state):
        out = self.mlp(state)
        action_scores = self.actor(out)
        state_value = self.critic(out)
        return action_scores, state_value

    def compute_action(self, state, action_std):
        state = torch.from_numpy(state).float().unsqueeze(0)
        probs, state_value = self(state)
        probs = torch.tanh(probs)

        action_var = torch.full((self.act_len,), action_std * action_std)
        cov_mat = torch.diag(action_var).unsqueeze(dim=0)
      
        m = torch.distributions.multivariate_normal.MultivariateNormal(probs, cov_mat)
        
        action = m.sample()

        action_clamped = torch.tanh(action)

        action_clamped[0][0] = action_clamped[0][0]*0.6-0.1
        action_clamped[0][3] = action_clamped[0][3]*0.6+0.1
        action_clamped[0][6] = action_clamped[0][6]*0.6-0.1
        action_clamped[0][9] = action_clamped[0][9]*0.6+0.1
      
        return action_clamped.detach().numpy(), m.log_prob(action_clamped).detach().numpy(), state_value.detach()

# Environment

In [ ]:
class Env():
    def __init__(self):
      #Select here the model you want to train: scene.xml or scene3.xml to train with a step on the floor
      self.model = mujoco.MjModel.from_xml_path("./MUJOCO-AIDL/anybotics_anymal_c/scene.xml")
      self.data = mujoco.MjData(self.model)
      self.renderer=mujoco.Renderer(self.model)
      mujoco.mj_kinematics(self.model, self.data)
      mujoco.mj_forward(self.model, self.data)
      self.FRAMERATE = 60 #Hz
      self.DURATION = 8 #Seconds
      self.TIMESTEP = 0.002 # 0.002 By Default
      self.done = False
      self.model.opt.timestep = self.TIMESTEP
      # Make a new camera, move it to a closer distance.
      self.camera = mujoco.MjvCamera()
      mujoco.mjv_defaultFreeCamera(self.model, self.camera)
      self.camera.distance = 5
      self.frames=[]

    def reset(self):
      mujoco.mj_resetDataKeyframe(self.model, self.data, 0)
      self.data.ctrl=np.zeros(12)
      state= np.array(self.data.qpos.copy())
      state= np.append(state, self.data.qvel.copy())
      self.frames.clear()
      return state

    def step(self, action, render=False):
      self.done=False
      reward = 0
      self.data.ctrl=action
      for i in range(MUJOCO_STEPS):
        mujoco.mj_step(self.model, self.data)
        reward = reward + self.data.qvel[0] + (self.data.qpos[2]-0.5)
        if render and (len(self.frames) < self.data.time * self.FRAMERATE):
          self.camera.lookat = self.data.body('LH_SHANK').subtree_com
          self.renderer.update_scene(self.data, self.camera)
          pixels = self.renderer.render()
          self.frames.append(pixels.copy())

      state= np.array(self.data.qpos.copy())
      state= np.append(state, self.data.qvel.copy())
      if self.data.time > self.DURATION:
        self.done = True
      if self.data.qpos[2] < 0.45:
        self.done = True
        reward = reward - 100
      return state, reward, self.done

    def close(self,episode,ep_reward):
      path=f'./video_{episode}_{ep_reward}.mp4'
      media.write_video(path, self.frames, fps=self.FRAMERATE)

# Replay memory

In [ ]:
transition = np.dtype([('s', np.float64, (37,)), ('a', np.float64, (12,)), ('a_logp', np.float64, (12,)),
                       ('r', np.float64), ('s_', np.float64, (37,))])

class ReplayMemory():
    def __init__(self, capacity):
        self.buffer_capacity = capacity
        self.buffer = np.empty(capacity, dtype=transition)
        self.counter = 0

    # Stores a transition and returns True or False depending on whether the buffer is full or not
    def store(self, transition):
        self.buffer[self.counter] = transition
        self.counter += 1
        if self.counter == self.buffer_capacity:
            self.counter = 0
            return True
        else:
            return False

# Train and Test Function

In [ ]:
def train(policy, optimizer, memory, hparams, action_std):

    gamma = hparams['gamma']
    ppo_epoch = hparams['ppo_epoch']
    batch_size = hparams['batch_size']
    clip_param = hparams['clip_param']
    c1 = hparams['c1']
    c2 = hparams['c2']


    s = torch.tensor(memory.buffer['s'], dtype=torch.float)
    a = torch.tensor(memory.buffer['a'], dtype=torch.float)
    r = torch.tensor(memory.buffer['r'], dtype=torch.float).view(-1, 1)
    s_ = torch.tensor(memory.buffer['s_'], dtype=torch.float)

    old_a_logp = torch.tensor(memory.buffer['a_logp'], dtype=torch.float).view(-1, 1)
    action_var = torch.full((12,), action_std * action_std)
    cov_mat = torch.diag(action_var).unsqueeze(dim=0)

    with torch.no_grad():
        target_v = r + gamma * policy(s_)[1]
        adv = target_v - policy(s)[1]

    for _ in range(ppo_epoch):
        for index in BatchSampler(SubsetRandomSampler(range(memory.buffer_capacity)), batch_size, False):
            probs, _ = policy(s[index])
            dist = MultivariateNormal(probs,cov_mat)
            entropy = dist.entropy()
            
            a_logp = dist.log_prob(a[index]).unsqueeze(dim=1)

            ratio = torch.exp(a_logp-old_a_logp[index])

            surr1 = ratio * adv[index]

            surr2 = torch.clamp(ratio,1-clip_param,1+clip_param) * adv[index]

            policy_loss = torch.min(surr1, surr2).mean()
            value_loss = F.smooth_l1_loss(policy(s[index])[1], target_v[index])
            entropy = entropy.mean()

            loss = -policy_loss+c1*value_loss-c2*entropy

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    return -policy_loss.item(), value_loss.item(), entropy.item(), ratio.mean().item()

def test(action_std ,env, policy, episode, render=False):
    state, ep_reward, done = env.reset(), 0, False
    counter=0
    reward_list=[]
    cumulative_reward_list=[]
    time_list=[]
    while not done:
        action, _, _ = policy.compute_action(state, action_std)
        state, reward, done = env.step(action, render=True)
        reward_list.append(reward)
        time_list.append(counter*0.002*MUJOCO_STEPS)
        ep_reward += reward
        cumulative_reward_list.append(ep_reward)
        counter = counter + 1

    #Saving Video and sending it to wandb
    env.close(episode,ep_reward)
    wandb.log({"Video eval": wandb.Video(f'./video_{episode}_{ep_reward}.mp4', fps=4, format="mp4")})

    #Ploting Episode Reward
    plt.plot(time_list, reward_list)
    plt.plot(time_list, cumulative_reward_list)
    plt.xlabel('Time (seconds)')
    plt.ylabel('Reward')
    plt.title('Episode instant and cumulative Reward')

    wandb.log({"Reward eval": plt})

    return ep_reward

# Hyperparameters

In [ ]:
hparams = {
      'gamma' : 0.99,
      'log_interval' : 50,
      'num_episodes': 15000,
      'lr' : 1e-5,
      'clip_param': 0.1,
      'ppo_epoch': 48,
      'replay_size': 6400,
      'batch_size': 128,
      'c1': 1.,
      'c2': 0.001,
      'std_init': 1.0,
      'std_min': 0.6,
      }

#Define the train or sweep function

In [ ]:
def train_or_sweep(is_sweep=True):
    
    wandb.init(project=PROJECT)        

    if is_sweep:
      hparams.update(wandb.config)
      print('Params updated')
      
    # Create environment
    env = Env()

    # Fix random seed (for reproducibility)
    seed=0
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

    # Set number of imputs and actions
    n_inputs = 37
    n_actions = 12

    # Create policy and optimizer
    policy = Agent(n_inputs, n_actions)
    optimizer = torch.optim.Adam(policy.parameters(), lr=hparams['lr'])
    memory = ReplayMemory(hparams['replay_size'])
    
    # Load here your model and policy to start your train from a saved checkpoint
    #policy = torch.load('./volcanic-star-9_12869_Reward-4092.94_policy.pt')
    #optimizer = torch.load('./volcanic-star-9_12869_Reward-4092.94_optimizer.pt')

    #Initialize the action_std, decay and init for the covariance matrix of the distribution
    action_std_decay = -(hparams['std_min']-hparams['std_init'])*hparams['log_interval']/hparams['num_episodes']
    action_std_init = hparams['std_init']
    action_std = action_std_init

    #Define the target_reward to stop the run before reaching the num_episodes value
    target_reward=10000
    print(f"Target reward: {target_reward}")

    # Training loop
    running_reward = -100
    saving_reward = 0
    for i_episode in range(hparams['num_episodes']):
        # Collect experience
        state, ep_reward, done = env.reset(), 0, False

        while not done:  # Don't infinite loop while learning
            action, a_logp, state_value = policy.compute_action(state, action_std)
            next_state, reward, done = env.step(action, render=False)

            if memory.store((state, action, a_logp, reward, next_state)):
                policy_loss, value_loss, avg_entropy, ratio = train(policy, optimizer, memory, hparams, action_std)
                wandb.log(
                    {
                    'policy_loss': policy_loss,
                    'value_loss': value_loss,
                    'avg_reward': running_reward,
                    'avg_entropy': avg_entropy,
                    'ratio': ratio
                    })

            state = next_state
            ep_reward += reward

            if done:
                break

        # Update running reward
        running_reward = round(0.05 * ep_reward + (1 - 0.05) * running_reward,2)
        
        #Log to check episode rewards and std
        if i_episode % hparams['log_interval'] == 0:
            print(f'Episode {i_episode}\tLast reward: {ep_reward:.2f}\tAverage reward: {running_reward:.2f}\tAction standard deviation: {action_std:.5f}')
            action_std = action_std - action_std_decay
            action_std = round(action_std, 5)
        
        #Define the minimum reward needed to save a model and make a video
        if running_reward > 2000:
            if running_reward > saving_reward:
                saving_reward = running_reward
                torch.save(policy, f'./{wandb.run.name}_{i_episode}_Reward-{running_reward}_policy.pt')
                torch.save(optimizer, f'./{wandb.run.name}_{i_episode}_Reward-{running_reward}_optimizer.pt')
                wandb.save(f'./{wandb.run.name}_{i_episode}_Reward-{running_reward}_policy.pt')
                wandb.save(f'./{wandb.run.name}_{i_episode}_Reward-{running_reward}_optimizer.pt')
                print(f'Policy and Optimizer have been saved')
                ep_reward = test(action_std, env, policy,i_episode)
                print(f'Episode {i_episode} Video reward: {ep_reward}')

        if running_reward > target_reward:
            print("Solved!")
            torch.save(policy, f'./{wandb.run.name}_{i_episode}_Reward-{running_reward}_policy.pt')
            torch.save(optimizer, f'./{wandb.run.name}_{i_episode}_Reward-{running_reward}_optimizer.pt')
            wandb.save(f'./{wandb.run.name}_{i_episode}_Reward-{running_reward}_policy.pt')
            wandb.save(f'./{wandb.run.name}_{i_episode}_Reward-{running_reward}_optimizer.pt')
            ep_reward = test(action_std, env, policy,i_episode)
            print(f'Episode {i_episode} Video reward: {ep_reward}')

            break

    print(f"Finished training! Running reward is now {running_reward}")

#Execute this to train with the specified hparams

In [ ]:
train_or_sweep(is_sweep=False)

# Hyperparameter sweeping
Here you can run a sweep for Hyperparameters and fine tune the longer training.

A Higher number of episodes will help do better sweeps but it will take considerably longer to do.

In [ ]:
sweep_configuration = {
    "name": f"ppo_sweep_0",
    "method": 'bayes',
    "metric": {
        "name": "avg_reward",
        "goal": "maximize"
    },
    "parameters": {
        "lr": {
          "distribution": "uniform",
          "max": 0.0001,
          "min": 0.00001
        },
        "ppo_epoch": {
          "distribution": "int_uniform",
          "max": 60,
          "min": 40
        },
        "c2": {
          "distribution": "uniform",
          "max": 0.01,
          "min": 0.001
        },
        "replay_size": {
          "distribution": "int_uniform",
          "max": 10000.,
          "min": 6000.
        },
        "std_init": {
          "distribution": "uniform",
          "max": 1.1,
          "min": 1.0
        },
        "std_min": {
          "distribution": "uniform",
          "max": 0.8,
          "min": 0.7
        }
    }
  }

In [ ]:
sweep_id = wandb.sweep(sweep=sweep_configuration, project=PROJECT)
wandb.agent(sweep_id, function=train_or_sweep, count=50, project=PROJECT)